## CIFAR10 DATASET SPLIT ##

Total_Trainset = 50000, Total_Testset = 10000

### Centralized ###

Trainset = 45000, Valset= 5000, Testset = 10000

### Federated ###

NUM_CLIENTS = 10
Trainset = 4500, Valset= 500, (Each client)
Testset = 10000 (Aggregated Model)

metrics_distributed 가 각 client에서 돌린 결과 평균낸 accuracy
metrics_centralized 가 모은 model test한 결과 나온 accuracy

In [1]:
from collections import OrderedDict
from typing import List, Tuple, Dict, Optional

import matplotlib.pyplot as plt
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import DataLoader, random_split
from torchvision.datasets import CIFAR10
import torchvision.models

from sklearn.metrics import f1_score, precision_score, recall_score

import os
# 여기서는 10개로 나눈 것 중, 하나만 train 함.
import pandas as pd


import flwr as fl
from flwr.common import Metrics
import flwr.common

In [2]:

DEVICE = torch.device("cuda")  # Try "cuda" to train on GPU
print(
    f"Training on {DEVICE} using PyTorch {torch.__version__} and Flower {fl.__version__}"
)

NUM_CLIENTS = 10
EPOCH = 100
BATCH_SIZE = 32

client_resources = None
if DEVICE.type == "cuda":
    client_resources = {"num_gpus": 1}

transform = transforms.Compose(
    [transforms.ToTensor(), 
     transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010))]
)
trainset = CIFAR10("../dataset", train=True, download=True, transform=transform)
testset = CIFAR10("../dataset", train=False, download=True, transform=transform)

# Split dataset
lengths = [45000, 5000]
split_trainset, valset = random_split(trainset, lengths, torch.Generator().manual_seed(42)) 
    
# 45000
full_split_trainloader = DataLoader(split_trainset, batch_size=BATCH_SIZE, shuffle=True)
# 5000
full_valloader = DataLoader(valset, batch_size=BATCH_SIZE, shuffle=True)
# 10000
full_testloader = DataLoader(testset, batch_size=BATCH_SIZE, shuffle=True)


def load_datasets():
    # Split training set into 10 partitions to simulate the individual dataset
    partition_size = len(trainset) // NUM_CLIENTS
    lengths = [partition_size] * NUM_CLIENTS
    datasets= random_split(trainset, lengths, torch.Generator().manual_seed(42))

    # Split each partition into train/val and create DataLoader
    trainloaders = []
    valloaders = []
    for idx, ds in enumerate(datasets):
        len_val = len(ds) // 10  # 10 % validation set
        len_train = len(ds) - len_val
        lengths = [len_train, len_val]
        ds_train, ds_val = random_split(ds, lengths, torch.Generator().manual_seed(42))
        # Always splits in the same way.

        # Count Distribution of data
        # print(dict(Counter(ds_train.targets)))
        # custom_subset(ds_train)
        # arr = []
        # print("ds_train", idx)
        # for a in ds_train:
        #     arr.append(a[1])
        # for i in range(10):
        #     print(i, ":", arr.count(i))
        # Data is not perfectly distributed

        trainloaders.append(DataLoader(ds_train, batch_size=BATCH_SIZE, shuffle=True))
        valloaders.append(DataLoader(ds_val, batch_size=BATCH_SIZE))
    testloader = DataLoader(testset, batch_size=BATCH_SIZE)
    return trainloaders, valloaders, testloader

trainloaders, valloaders, testloader = load_datasets()


Training on cuda using PyTorch 2.0.0 and Flower 1.4.0
Files already downloaded and verified
Files already downloaded and verified


### VGG 16 ###

In [3]:
# all_models = torchvision.models.list_models()
# all_models

In [4]:
# net = torchvision.models.densenet121(num_classes=10).to(DEVICE)
# net = torchvision.models.resnet18(num_classes=10).to(DEVICE)
# net = torchvision.models.resnet34(num_classes=10).to(DEVICE)
# net = torchvision.models.resnet50(num_classes=10).to(DEVICE)

# net = torchvision.models.googlenet(num_classes=10).to(DEVICE)
# net

In [5]:
def train(net, trainloader, epochs: int, verbose=False):
    """Train the network on the training set."""
    criterion = torch.nn.CrossEntropyLoss()
    optimizer = torch.optim.SGD(net.parameters(), lr=0.01, momentum=0.5)
    net.train()
    for epoch in range(epochs):
        correct, total, epoch_loss = 0, 0, 0.0
        for images, labels in trainloader:
            images, labels = images.to(DEVICE), labels.to(DEVICE)
            optimizer.zero_grad()
            outputs = net(images)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            # Metrics
            epoch_loss += loss
            total += labels.size(0)
            correct += (torch.max(outputs.data, 1)[1] == labels).sum().item()
        epoch_loss /= len(trainloader.dataset)
        epoch_acc = correct / total
        if verbose:
            print(f"Epoch {epoch+1}: train loss {epoch_loss}, accuracy {epoch_acc}")


def test(net, testloader):
    """Evaluate the network on the entire test set."""
    criterion = torch.nn.CrossEntropyLoss()
    correct, total, loss = 0, 0, 0.0
    pred = torch.Tensor([])
    target = torch.Tensor([])
    net.eval()
    with torch.no_grad():
        for images, labels in testloader:
            images, labels = images.to(DEVICE), labels.to(DEVICE)
            outputs = net(images)
            loss += criterion(outputs, labels).item()
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
            pred = torch.cat([pred, predicted.cpu()], dim=0)
            target = torch.cat([target, labels.cpu()], dim=0)
    loss /= len(testloader.dataset)
    accuracy = correct / total
    recall = recall_score(pred, target, average='macro')
    precision = precision_score(pred, target, average='macro')
    f1 = f1_score(pred, target, average='macro')
    return loss, {"loss": loss, "accuracy": accuracy, "recall": recall, "precision": precision, "f1_score": f1}


## Federated ##

In [6]:
def get_parameters(net) -> List[np.ndarray]:
    return [val.cpu().numpy() for _, val in net.state_dict().items()]

def set_parameters(net, parameters: List[np.ndarray]):
    params_dict = zip(net.state_dict().keys(), parameters)
    for k, v in params_dict:
        continue
    state_dict = OrderedDict({k: torch.Tensor(v) for k, v in params_dict})

    net.load_state_dict(state_dict, strict=False)

class FlowerClient(fl.client.NumPyClient):
    def __init__(self, net, trainloader, valloader):
        self.net = net
        self.trainloader = trainloader
        self.valloader = valloader

    def get_parameters(self, config):
        return get_parameters(self.net)

    def fit(self, parameters, config):
        set_parameters(self.net, parameters)
        train(self.net, self.trainloader, epochs=1)
        return get_parameters(self.net), len(self.trainloader), {}

    def evaluate(self, parameters, config):
        set_parameters(self.net, parameters)
        loss, metrics = test(self.net, self.valloader)
        return float(loss), len(self.valloader), metrics
    
def client_fn(cid: str) -> FlowerClient:
    """Create a Flower client representing a single organization."""

    # Load model
    # net = Net().to(DEVICE)

    # Load data (CIFAR-10)
    # Note: each client gets a different trainloader/valloader, so each client
    # will train and evaluate on their own unique data
    trainloader = trainloaders[int(cid)]
    valloader = valloaders[int(cid)]

    # Create a  single Flower client representing a single organization
    return FlowerClient(net, trainloader, valloader)
    
def weighted_average(metrics: List[Tuple[int, Metrics]]) -> Metrics:

    averages = {}
    targets = ["accuracy", "recall", "precision", "f1_score"]

    total_examples = sum([num_examples for num_examples, _ in metrics])
    for target in targets:
        target_distributed = [num_examples * m[target] for num_examples, m in metrics]
        averages[target] = sum(target_distributed) / total_examples
    
    # Aggregate and return custom metric (weighted average)
    return averages

# The `evaluate` function will be by Flower called after every round
def evaluate(
    server_round: int,
    parameters: fl.common.NDArrays,
    config: Dict[str, fl.common.Scalar],
) -> Optional[Tuple[float, Dict[str, fl.common.Scalar]]]:
    # net = Net().to(DEVICE)
    set_parameters(net, parameters)  # Update model with the latest parameters
    loss, metrics = test(net, full_testloader) # Test total test_loader
    print(f"Server-side evaluation loss {loss} / accuracy {metrics['accuracy']}")
    return loss, metrics


In [7]:
models = {
    'Densenet121' : "densenet121",
    'Mobilenet_v2': "mobilenet_v2",
    'Resnet18' : "resnet18",
    'Resnet34' : "resnet34",
    'Resnet50' : "resnet50",
    'VGG11' : "vgg11", 
    'VGG13' : "vgg13", 
    'VGG16' : "vgg16", 
    'VGG19' : "vgg19",
    # 'Googlenet' : "googlenet" # fails
}

In [8]:
df_final = pd.DataFrame()

for key, value in models.items():
    net = getattr(torchvision.models, value)(num_classes=10).to(DEVICE)
    ######## Centralized #########
    print("Centralized: " + key)
    trained_path = "../dataset/trained_centralized_"+key+".pkl"
    df_centralized = pd.DataFrame()
    for epoch in range(EPOCH):
        train(net, full_split_trainloader, 1)
        loss, c_metrics = test(net, full_testloader) # Federated aggregation model tests full_testloader
        print(f"{key} Epoch {epoch+1}: validation loss {loss}, accuracy {c_metrics['accuracy']}")
        
        df_centralized['round'] = epoch+1,
        df_centralized['Centralized'] = 'Centralized',
        df_centralized['model'] = key
        for metric in c_metrics:
            df_centralized[f"c_{metric}"] = c_metrics[metric]
        df_final = pd.concat([df_final, df_centralized], axis=0)
    df_centralized.to_csv('../result/result_c_'+key+'.csv', index=False)
    torch.save(net.state_dict(), trained_path)

    ######## Federated ######## 
    # Reset net
    net = getattr(torchvision.models, value)(num_classes=10).to(DEVICE) 
    print("Federated: " + key)
    fedavg = fl.server.strategy.FedAvg(
        fraction_fit=1.0,
        fraction_evaluate=0.5,
        min_fit_clients=10,
        min_evaluate_clients=5,
        min_available_clients=10,
        evaluate_metrics_aggregation_fn=weighted_average,   # aggregate evaluation of local model
        evaluate_fn=evaluate,   # evaluate global model
    )  
    hist = fl.simulation.start_simulation(
        client_fn=client_fn,
        num_clients=NUM_CLIENTS,
        config=fl.server.ServerConfig(num_rounds=EPOCH),
        strategy=fedavg,
        client_resources=client_resources,
    ) 

    df_federated = pd.DataFrame()
    df_federated['round'] = [i for i in range(1, EPOCH + 1)]
    df_federated['Centralized'] = "Federated" 
    df_federated['model'] = key

    # centralized metrics
    metrics_cen = list(hist.metrics_centralized.keys())
    metrics_dis = list(hist.metrics_distributed.keys())

    for metric in metrics_cen:
        df_federated[f"c_{metric}"] = [h[1] for h in hist.metrics_centralized[metric][1:]]
    for metric in metrics_dis:
        df_federated[f"d_{metric}"] = [h[1] for h in hist.metrics_distributed[metric]]
    
    df_federated.to_csv('../result/result_f_'+key+'.csv', index=False)

    df_final = pd.concat([df_final, df_federated], axis=0)

df_final.to_csv('../result/result_total.csv', index=False)

Centralized: Mobilenet_v2


INFO flwr 2023-05-05 02:33:31,319 | app.py:146 | Starting Flower simulation, config: ServerConfig(num_rounds=1, round_timeout=None)


Mobilenet_v2 Epoch 1: validation loss 0.05145465053319931, accuracy 0.396
Federated: Mobilenet_v2


2023-05-05 02:33:33,085	INFO worker.py:1553 -- Started a local Ray instance.
INFO flwr 2023-05-05 02:33:33,623 | app.py:180 | Flower VCE: Ray initialized with resources: {'memory': 77204689511.0, 'node:172.17.0.2': 1.0, 'CPU': 16.0, 'accelerator_type:RTX': 1.0, 'GPU': 1.0, 'object_store_memory': 37373438361.0}
INFO flwr 2023-05-05 02:33:33,623 | server.py:86 | Initializing global parameters
INFO flwr 2023-05-05 02:33:33,624 | server.py:273 | Requesting initial parameters from one random client
INFO flwr 2023-05-05 02:33:35,603 | server.py:277 | Received initial parameters from one random client
INFO flwr 2023-05-05 02:33:35,603 | server.py:88 | Evaluating initial parameters
/home/dongha/anaconda3/envs/fl/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
INFO f

Server-side evaluation loss 0.072070906996727 / accuracy 0.1
(autoscaler +45s) Tip: use `ray status` to view detailed cluster status. To disable these messages, set RAY_SCHEDULER_EVENTS=0.
(autoscaler +45s) Warning: The following resource request cannot be scheduled right now: {'GPU': 1.0, 'CPU': 1.0}. This is likely due to all cluster resources being claimed by actors. Consider creating fewer actors or adding more nodes to this Ray cluster.


DEBUG flwr 2023-05-05 02:34:16,898 | server.py:232 | fit_round 1 received 10 results and 0 failures
WARNING flwr 2023-05-05 02:34:17,226 | fedavg.py:243 | No fit_metrics_aggregation_fn provided
/home/dongha/anaconda3/envs/fl/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
INFO flwr 2023-05-05 02:34:19,147 | server.py:119 | fit progress: (1, 0.072070906996727, {'loss': 0.072070906996727, 'accuracy': 0.1, 'recall': 0.018577450249469517, 'precision': 0.1, 'f1_score': 0.018766503286116884}, 41.64961243001744)
DEBUG flwr 2023-05-05 02:34:19,147 | server.py:168 | evaluate_round 1: strategy sampled 5 clients (out of 10)


Server-side evaluation loss 0.072070906996727 / accuracy 0.1


(launch_and_evaluate pid=1443537) /home/dongha/anaconda3/envs/fl/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
(launch_and_evaluate pid=1443537)   _warn_prf(average, modifier, msg_start, len(result))
(launch_and_evaluate pid=1443597) /home/dongha/anaconda3/envs/fl/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
(launch_and_evaluate pid=1443597)   _warn_prf(average, modifier, msg_start, len(result))
(launch_and_evaluate pid=1443666) /home/dongha/anaconda3/envs/fl/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zer

['loss', 'accuracy', 'recall', 'precision', 'f1_score']
['accuracy', 'recall', 'precision', 'f1_score']


(launch_and_evaluate pid=1443802) /home/dongha/anaconda3/envs/fl/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
(launch_and_evaluate pid=1443802)   _warn_prf(average, modifier, msg_start, len(result))
